In [ ]:
import pandas as pd
from habanero import Crossref
from concurrent.futures import ThreadPoolExecutor


# Función para obtener el número de resultados usando la API de CrossRef
def get_results_count(query):
    cr = Crossref()
    result = cr.works(query=query, rows=0)  # Rows=0 devuelve solo el número total de resultados
    return result['message']['total-results']

# Función para procesar una única combinación de tipos de tumor y fármacos
def process_combination(tipo_tumor, farmaco):
    query = f"{tipo_tumor} AND {farmaco}"
    results_count = get_results_count(query)
    return {'TipoTumor': tipo_tumor, 'Farmaco': farmaco, 'Resultados': results_count}


# Cargar tipos de tumor y fármacos desde CSV sin encabezado
tipos_de_tumor_df = pd.read_csv('tipo_tumores.csv', header=None, names=['TipoTumor'])
farmacos_df = pd.read_csv('farmacos.csv', header=None, names=['Farmaco'])

In [3]:
# Inicializar una lista para almacenar los resultados
results_list = []

# Bucle anidado para combinar tipos de tumor y fármacos
for tipo_tumor in tipos_de_tumor_df['TipoTumor']:
    for farmaco in farmacos_df['Farmaco']:
        # Crear la consulta combinando el tipo de tumor y el fármaco
        query = f"{tipo_tumor} AND {farmaco}"
        # Obtener el número de resultados
        results_count = get_results_count(query)
        # Agregar los resultados a la lista
        results_list.append({'TipoTumor': tipo_tumor, 'Farmaco': farmaco, 'Resultados': results_count})

# Crear un DataFrame a partir de la lista de resultados
results_df = pd.DataFrame(results_list)

# Mostrar el DataFrame final
print("DataFrame de Resultados:")
print(results_df)

In [ ]:
# Crear una lista de combinaciones de tipos de tumor y fármacos
combinations = [(tipo_tumor, farmaco) for tipo_tumor in tipos_de_tumor_df['TipoTumor'] for farmaco in farmacos_df['Farmaco']]

# Utilizar ThreadPoolExecutor para procesar las combinaciones en paralelo
with ThreadPoolExecutor() as executor:
    results_list = list(executor.map(lambda x: process_combination(*x), combinations))

# Crear un DataFrame a partir de la lista de resultados
results_df = pd.DataFrame(results_list)

# Mostrar el DataFrame final
print("DataFrame de Resultados:")
print(results_df)